In [1]:
import numpy as np
import tensorflow as tf
from keras import Model, Input
from keras.layers import Conv2D, Activation, MaxPool2D, Flatten, Dense, BatchNormalization
BATCH=2

2024-01-17 19:38:20.500493: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-17 19:38:20.500528: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-17 19:38:20.501681: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-17 19:38:20.507300: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-17 19:38:21.207623: W tensorflow/compiler/tf2

In [3]:
class MyNet(Model):
    def __init__(self, num_of_actions):
        '''These are for the generalization of the function change2To3(new_net, old_net)'''
        self.c3_1 = None
        self.c2_1 = None
        super(MyNet, self).__init__()
        self.num_of_actions = num_of_actions
        self.b1 = BatchNormalization(name='batch1')  # BN层
        self.c1_1 = Conv2D(filters=32, kernel_size=(3, 3), padding='same', name='conv_1',
                           kernel_initializer=tf.keras.initializers.TruncatedNormal(mean=0.0, stddev=0.01, seed=None),
                           bias_initializer = tf.keras.initializers.Constant(value=0.01))  # 卷积层
        self.b0 = BatchNormalization(name='batch0')  # BN层
        self.a1_1 = Activation('relu', name='relu_1')  # 激活层
        self.p1 = MaxPool2D(pool_size=(2, 2), strides=2, padding='same', name='padding_1')  # 池化层
        #self.d1 = Dropout(0.2)  # dropout层

        self.flatten = Flatten()
        self.f1 = Dense(512, activation='relu', name='dense1',
                           kernel_initializer=tf.keras.initializers.TruncatedNormal(mean=0.0, stddev=0.01, seed=None),
                           bias_initializer = tf.keras.initializers.Constant(value=0.01))
        self.f2 = Dense(num_of_actions, activation=None, name='dense2',
                           kernel_initializer=tf.keras.initializers.TruncatedNormal(mean=0.0, stddev=0.01, seed=None),
                           bias_initializer = tf.keras.initializers.Constant(value=0.01))

    def call(self, x):
        #print(x.shape)
        x = self.b1(x)
        x = self.c1_1(x)
        #print(x.shape)
        x = self.b0(x)
        x = self.a1_1(x)
        x = self.p1(x)

        x = self.flatten(x)
        x = self.f1(x)
        y = self.f2(x)
        return y


class MyNet2(Model):
    def __init__(self, num_of_actions):
        '''These are for the generalization of the function change2To3(new_net, old_net)'''
        self.c3_1 = None
        super(MyNet2, self).__init__()
        self.b2 = BatchNormalization(name='batch2')  # BN层
        self.num_of_actions = num_of_actions
        self.conv2_num_of_filters = 32
        self.c2_1 = Conv2D(filters=self.conv2_num_of_filters, kernel_size=(3, 3), padding='same', name='conv_2',
                           kernel_initializer=tf.keras.initializers.TruncatedNormal(mean=0.0, stddev=0.01, seed=None),
                           bias_initializer = tf.keras.initializers.Constant(value=0.01))  # 卷积层
        self.b1 = BatchNormalization(name='batch1')  # BN层
        self.a2_1 = Activation('relu', name='relu_2')  # 激活层

        self.p2 = MaxPool2D(pool_size=(2, 2), strides=2, padding='same', name='padding_2')  # 池化层
        self.c1_1 = Conv2D(filters=32, kernel_size=(3, 3), padding='same', name='conv_1',
                           kernel_initializer=tf.keras.initializers.TruncatedNormal(mean=0.0, stddev=0.01, seed=None),
                           bias_initializer = tf.keras.initializers.Constant(value=0.01))  # 卷积层
        self.b0 = BatchNormalization(name='batch0')  # BN层
        self.a1_1 = Activation('relu', name='relu_1')  # 激活层
        self.p1 = MaxPool2D(pool_size=(2, 2), strides=2, padding='same', name='padding_1')  # 池化层
        #self.d1 = Dropout(0.2)  # dropout层

        self.flatten = Flatten()
        self.f1 = Dense(512, activation='relu', name='dense1',
                           kernel_initializer=tf.keras.initializers.TruncatedNormal(mean=0.0, stddev=0.01, seed=None),
                           bias_initializer = tf.keras.initializers.Constant(value=0.01))
        self.f2 = Dense(num_of_actions, activation=None, name='dense2',
                           kernel_initializer=tf.keras.initializers.TruncatedNormal(mean=0.0, stddev=0.01, seed=None),
                           bias_initializer = tf.keras.initializers.Constant(value=0.01))
    def call(self, x):
        x = self.b2(x)
        x = self.c2_1(x)
        x = self.b1(x)
        x = self.a2_1(x)
        x = self.p2(x)
        x = self.c1_1(x)
        x = self.b0(x)
        x = self.a1_1(x)
        x = self.p1(x)

        x = self.flatten(x)
        x = self.f1(x)
        y = self.f2(x)
        return y
    def load_stage1(self, stage1_net):
        new_kernel = custom_kernel_stage2(stage1_net, self.conv2_num_of_filters // 4)
        self.c1_1.set_weights([new_kernel, stage1_net.c1_1.get_weights()[1]])
        self.f1.set_weights([stage1_net.f1.get_weights()[0], stage1_net.f1.get_weights()[1]])
        self.f2.set_weights(stage1_net.f2.get_weights())

In [4]:
net1 = MyNet(2)
input_sidelength = 40
net1.build(input_shape=(BATCH, input_sidelength, input_sidelength, 4))
net1.call(Input(shape=(input_sidelength, input_sidelength, 4)))
net1.load_weights('model/FlappyBird.h5', by_name=True)
net1.summary()

Model: "my_net"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch1 (BatchNormalization  (None, 40, 40, 4)         16        
 )                                                               
                                                                 
 conv_1 (Conv2D)             (None, 40, 40, 32)        1184      
                                                                 
 batch0 (BatchNormalization  (None, 40, 40, 32)        128       
 )                                                               
                                                                 
 relu_1 (Activation)         (None, 40, 40, 32)        0         
                                                                 
 padding_1 (MaxPooling2D)    (None, 20, 20, 32)        0         
                                                                 
 flatten (Flatten)           (None, 12800)             0    

2024-01-17 19:38:34.603194: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-17 19:38:34.638140: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-17 19:38:34.677146: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [ ]:
net2 = MyNet2(3)
input_sidelength = 80
net2.build(input_shape=(BATCH, input_sidelength, input_sidelength, 4))
net2.call(Input(shape=(input_sidelength, input_sidelength, 4)))
net2.load_weights('model/ControlGroup', by_name=True)
net2.summary()